In [1]:
import pandas as pd
import warnings
import sys
import numpy as np
import scipy as sc
import random
from scipy import stats
from numpy import log10, sqrt

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

hac_mo = '10'

## Load HAC file

In [2]:
cauti_file =  '/Users/kenlocey/GitHub/HACRP-HAIs/data/optimized_by_HAI_file_date/CAUTI/CAUTI_Data_opt_for_SIRs_2017_10.pkl'
clabsi_file = '/Users/kenlocey/GitHub/HACRP-HAIs/data/optimized_by_HAI_file_date/CLABSI/CLABSI_Data_opt_for_SIRs_2017_10.pkl'

mrsa_file = '/Users/kenlocey/GitHub/HACRP-HAIs/data/optimized_by_HAI_file_date/MRSA/MRSA_Data_opt_for_SIRs_2017_10.pkl'
cdi_file = '/Users/kenlocey/GitHub/HACRP-HAIs/data/optimized_by_HAI_file_date/CDI/CDI_Data_opt_for_SIRs_2017_10.pkl'


tdf, tdf1, tdf2 = 0, 0, 0
cauti_df, clabsi_df = 0, 0
mrsa_df, cdi_df = 0, 0

cauti_df = pd.read_pickle(cauti_file)
cauti_df['CAUTI SIS'] = (cauti_df['CAUTI Observed Cases'] - cauti_df['expected O'])/cauti_df['CAUTI Predicted Cases']
cauti_df.rename(columns={
        'O/E': 'CAUTI O/P',
        'simulated O': 'CAUTI simulated O',
        'simulated O/E': 'CAUTI simulated O/P',
        'expected O': 'CAUTI expected O',
        'expected O/E': 'CAUTI expected O/P',
        'pi_opt': 'CAUTI pi_opt',
        'z_opt': 'CAUTI z_opt',
    }, inplace=True)
cauti_sis_deciles = np.nanpercentile(cauti_df['CAUTI SIS'], np.arange(0, 100, 10))
cauti_rand_sir_deciles = np.nanpercentile(cauti_df['CAUTI expected O/P'], np.arange(0, 100, 10))

clabsi_df = pd.read_pickle(clabsi_file)
clabsi_df['CLABSI SIS'] = (clabsi_df['CLABSI Observed Cases'] - clabsi_df['expected O'])/clabsi_df['CLABSI Predicted Cases']
clabsi_df.rename(columns={
        'O/E': 'CLABSI O/P',
        'simulated O': 'CLABSI simulated O',
        'simulated O/E': 'CLABSI simulated O/P',
        'expected O': 'CLABSI expected O',
        'expected O/E': 'CLABSI expected O/P',
        'pi_opt': 'CLABSI pi_opt',
        'z_opt': 'CLABSI z_opt',
    }, inplace=True)
clabsi_sis_deciles = np.nanpercentile(clabsi_df['CLABSI SIS'], np.arange(0, 100, 10))
clabsi_rand_sir_deciles = np.nanpercentile(clabsi_df['CLABSI expected O/P'], np.arange(0, 100, 10))




mrsa_df = pd.read_pickle(mrsa_file)
mrsa_df['MRSA SIS'] = (mrsa_df['MRSA Observed Cases'] - mrsa_df['expected O'])/mrsa_df['MRSA Predicted Cases']
mrsa_df.rename(columns={
        'O/E': 'MRSA O/P',
        'simulated O': 'MRSA simulated O',
        'simulated O/E': 'MRSA simulated O/P',
        'expected O': 'MRSA expected O',
        'expected O/E': 'MRSA expected O/P',
        'pi_opt': 'MRSA pi_opt',
        'z_opt': 'MRSA z_opt',
    }, inplace=True)
tdf = mrsa_df[mrsa_df['MRSA Predicted Cases'] >= 1]
mrsa_sis_deciles = np.nanpercentile(tdf['MRSA SIS'], np.arange(0, 100, 10))    
mrsa_rand_sir_deciles = np.nanpercentile(mrsa_df['MRSA expected O/P'], np.arange(0, 100, 10))


cdi_df = pd.read_pickle(cdi_file)
cdi_df['CDI SIS'] = (cdi_df['CDI Observed Cases'] - cdi_df['expected O'])/cdi_df['CDI Predicted Cases']
cdi_df.rename(columns={
        'O/E': 'CDI O/P',
        'simulated O': 'CDI simulated O',
        'simulated O/E': 'CDI simulated O/P',
        'expected O': 'CDI expected O',
        'expected O/E': 'CDI expected O/P',
        'pi_opt': 'CDI pi_opt',
        'z_opt': 'CDI z_opt',
    }, inplace=True)
tdf = cdi_df[cdi_df['CDI Predicted Cases'] >= 1]
cdi_sis_deciles = np.nanpercentile(tdf['CDI SIS'], np.arange(0, 100, 10))    
cdi_rand_sir_deciles = np.nanpercentile(cdi_df['CDI expected O/P'], np.arange(0, 100, 10))


ls1 = list(cauti_df)
ls2 = list(clabsi_df)
ls = list(filter(lambda x:x in ls1, ls2))
df_yr = cauti_df.merge(clabsi_df, on=ls, how='outer')
    
ls1 = list(df_yr)
ls2 = list(mrsa_df)
ls = list(filter(lambda x:x in ls1, ls2))
df_yr = df_yr.merge(mrsa_df, on=ls, how='outer')

ls1 = list(df_yr)
ls2 = list(cdi_df)
ls = list(filter(lambda x:x in ls1, ls2))
df_yr = df_yr.merge(cdi_df, on=ls, how='outer')


del tdf
del cauti_df, clabsi_df
del mrsa_df, cdi_df

df_yr.sort_values(by=['Facility ID', 'Line 19'], inplace=True, ascending=False)
df_yr.drop_duplicates(subset=['Facility ID'], inplace=True, keep='first')

print(len(df_yr['Facility ID'].unique()), 'hospitals resulting from merging the above dataframes.')

df_yr.head()


3141 hospitals resulting from merging the above dataframes.


,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final",CAUTI O/P,CAUTI simulated O,CAUTI simulated O/P,CAUTI expected O,CAUTI expected O/P,CAUTI pi_opt,CAUTI z_opt,CAUTI SIS,CLABSI O/P,CLABSI simulated O,CLABSI simulated O/P,CLABSI expected O,CLABSI expected O/P,CLABSI pi_opt,CLABSI z_opt,CLABSI SIS,MRSA O/P,MRSA simulated O,MRSA simulated O/P,MRSA expected O,MRSA expected O/P,MRSA pi_opt,MRSA z_opt,MRSA SIS,CDI O/P,CDI simulated O,CDI simulated O/P,CDI expected O,CDI expected O/P,CDI pi_opt,CDI z_opt,CDI SIS
2626,712686,4877064.0,6231790.0,4,670103,2017,6231790.0,6248755.0,62488.0,6186267.0,62317.90,NaN,NaN,NaN,NaN,6302015.0,NaN,7012013.0,2015-12-31,8.5,2014-01-01,MEDICAL CENTER OF ALLIANCE,2017.0,TX,8.50,10,Yes,336.0,141.0,195.0,0.0,0.0,0.282,0.376,0.000000,0.000000,NaN,NaN,8.00,Yes,1.0,5.000000e-01,NaN,NaN,5.0,5.0,5.0,5.0,NaN,NaN,10.0,NaN,NaN,7.0,5.0,NaN,4446.0,3957.0,0.0,2.0,0.159,2.320,0.000000,0.862069,NaN,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25344.0,NaN,NaN,NaN,NaN,NaN,2017_10,TX,NaN,NaN,NaN,NaN,146.559343,37144.0,86255.333333,62488.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.862069,1.0,0.431034,0.359564,0.154985,0.000854,33239.462244,0.707084
2627,712685,10611218.0,12130044.0,5,670098,2017,12130044.0,12073860.0,120739.0,11953121.0,121300.44,NaN,NaN,NaN,NaN,6302015.0,NaN,7012013.0,2015-12-31,5.5,2014-01-01,RESOLUTE HEALTH HOSPITAL,2017.0,TX,5.50,10,No,1159.0,715.0,444.0,0.0,0.0,0.910,0.664,0.000000,0.000000,NaN,NaN,5.50,No,1.0,0.000000e+00,NaN,NaN,5.0,5.0,5.0,5.0,NaN,NaN,1.0,NaN,NaN,10.0,5.0,NaN,14410.0,12525.0,0.0,13.0,0.632,7.342,0.000000,1.770635,NaN,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2017_10,TX,NaN,NaN,NaN,NaN,NaN,NaN,129806.400000,120739.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [3]:
main_df = pd.read_pickle('~/GitHub/HACRP-HAIs/data/Compiled_HCRIS-HACRP-HAI-RAND/Compiled_HCRIS-HACRP-HAI-RAND.pkl')
main_df = main_df[main_df['file_year'] == '2017']
main_df.sort_values(by='Facility ID', inplace=True)
print('Initial no. of hospitals in main_df:', len(main_df['Facility ID'].unique()))

main_df = main_df[(~main_df['Total HAC Score'].isin([float("NaN"), np.nan]))]
print('No. of hospitals in main_df having real HAC scores:', len(main_df['Facility ID'].unique()))

main_df = main_df[main_df['State'] != 'MD']
print('No. of these hospitals not in Maryland:', len(main_df['Facility ID'].unique()))

main_df = main_df[(main_df['CAUTI Predicted Cases'].isin([np.nan, float('NaN')])) | (main_df['CAUTI Predicted Cases'] < 1)]
main_df = main_df[(main_df['CLABSI Predicted Cases'].isin([np.nan, float('NaN')])) | (main_df['CLABSI Predicted Cases'] < 1)]
main_df = main_df[(main_df['MRSA Predicted Cases'].isin([np.nan, float('NaN')])) | (main_df['MRSA Predicted Cases'] < 1)]
main_df = main_df[(main_df['CDI Predicted Cases'].isin([np.nan, float('NaN')])) | (main_df['CDI Predicted Cases'] < 1)]
main_df = main_df[~main_df['AHRQ PSI-90 Score'].isin([np.nan, float('NaN')])]

print('No. of these hospitals without legit predicted cases for CAUTI,  CLABSI, MRSA, and CDI, but having legit scores for PSI-90:', len(main_df['Facility ID'].unique()))

main_df.head()


Initial no. of hospitals in main_df: 3314
No. of hospitals in main_df having real HAC scores: 3249
No. of these hospitals not in Maryland: 3202
No. of these hospitals without legit predicted cases for CAUTI,  CLABSI, MRSA, and CDI, but having legit scores for PSI-90: 92


,RPT_REC_NUM,IPPS interim payment (E_A_HOS_C1_72),IPPS payment (E_A_HOS_C1_59),PRVDR_CTRL_TYPE_CD,Facility ID,file_year,Line 19,Reconstructed IPPS payment (pre HAC penalty),Reconstructed HAC penalty,Reconstructed IPPS payment (post HAC penalty),HAC penalty imputed from E_A_HOS_C1_59,IPPS payment (from RAND),AHRQ PSI-90 Score,CAUTI Score,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Start Date,End Date,Domain 2 Score,Start Date,Facility Name,Fiscal Year,State,Total HAC Score,file_month,Payment Reduction,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,CAUTI Observed Cases,CLABSI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,CAUTI derived score,CLABSI derived score,Total HAC Score (derived),Payment Reduction (derived),Payment Reduction Reproduced?,HAC delta,CAUTI delta,CLABSI delta,AHRQ PSI-90 Footnote,CAUTI Footnote,CLABSI Footnote,Domain 1 Score Footnote,Domain 2 Score Footnote,SSI Footnote,SSI Score,Total HAC Footnote,CDI Footnote,CDI Score,MRSA Footnote,MRSA Score,MRSA patient days,CDI patient days,MRSA Observed Cases,CDI Observed Cases,MRSA Predicted Cases,CDI Predicted Cases,MRSA derived SIR,CDI derived SIR,MRSA derived score,CDI derived score,CDI delta,MRSA delta,Payment Reduction Footnote,CAUTI W Z Score,CDI W Z Score,CLABSI W Z Score,Domain 1 Footnote,Domain 2 Footnote,MRSA W Z Score,PSI-90 Footnote,PSI-90 W Z Score,SSI W Z Score,CAUTI derived Winsorized SIR,CAUTI derived W Z Score,CLABSI derived Winsorized SIR,CLABSI derived W Z Score,MRSA derived Winsorized SIR,MRSA derived W Z Score,HAC penalty (E_A_HOS_C1_7099),CDI derived Winsorized SIR,CDI derived W Z Score,PSI-90 End Date,PSI-90 Start Date,HAC penalty (imputed from RAND),file date,STATE,Dollar difference in payments (RAND vs E_A_HOS_C1_59),% Difference in payments (RAND vs E_A_HOS_C1_59),% Error in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),Dollar difference in penalties (E_A_HOS_C1_7099 vs Imputed from RAND),% Error in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),Dollar difference in penalties (E_A_HOS_C1_7099 vs Reconstructed HAC penalty),avg_Reconstructed HAC penalty,"HAC penalty, final"
3335,697199,482638.0,470802.0,9,010051,2017,470802.0,587805.0,5878.0,581927.0,4708.02,NaN,6.0,NaN,NaN,6302015.0,6.0,7012013.0,2015-12-31,NaN,2014-01-01,GREENE COUNTY HOSPITAL,2017.0,AL,6.0,10,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,6.0,No,1.0,0.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,5.0,NaN,NaN,5.0,NaN,5.0,NaN,2059.0,2079.0,0.0,2.0,0.074,0.927,0.0,2.157497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2017_10,AL,NaN,NaN,NaN,NaN,NaN,NaN,8093.285714,5878.0
3341,691609,1716819.0,1653004.0,5,010052,2017,1653004.0,2018609.0,20186.0,1998423.0,16530.04,NaN,6.0,NaN,NaN,6302015.0,6.0,7012013.0,2015-12-31,NaN,2014-01-01,LAKE MARTIN COMMUNITY HOSPITAL,2017.0,AL,6.0,10,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,6.0,No,1.0,0.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,5.0,NaN,NaN,5.0,NaN,5.0,NaN,2449.0,1682.0,0.0,0.0,0.088,0.690,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2017_10,AL,NaN,NaN,NaN,NaN,NaN,NaN,17959.571429,20186.0
3460,689817,868391.0,823403.0,13,010095,2017,823403.0,1025284.0,10253.0,1015031.0,8234.03,NaN,6.0,NaN,NaN,6302015.0,6.0,7012013.0,2015-12-31,NaN,2014-01-01,HALE COUNTY HOSPITAL,2017.0,AL,6.0,10,No,0.0,0.0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,6.0,No,1.0,0.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,5.0,NaN,NaN,5.0,NaN,5.0,NaN,1936.0,1936.0,0.0,0.0,0.069,0.817,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2017_10,AL,NaN,NaN,NaN,NaN,NaN,NaN,13301.142857,10253.0
3865,652804,382052.0,348123.0,7,030074,2017,348123.0,435154.0,4352.0,430802.0,3481.23,NaN,6.0,10.0,10.0,6302015.0,6.0,7012013.0,2015-12-31,10.0,2014-01-01,SELLS INDIAN HEALTH SERVICE HOSPITAL,2017.0,AZ,9.4,10,Yes,0.0,0.0,0.0,0.0,0.0,0.00

In [4]:
ls1 = list(df_yr)
ls2 = list(main_df)
ls = list(filter(lambda x:x in ls1, ls2))
df_yr = df_yr.merge(main_df, on=ls, how='outer')

print(df_yr.shape)
print(len(df_yr['Facility ID'].unique()), 'hospitals resulting from merging the above dataframes.')
print('This is NOT the same number as the number of hospitals having real HAC scores.')
print('Instead, it is the same as the number of hospitals with legit scores for PSI-90 or one or more HAIs.')


(3233, 132)
3233 hospitals resulting from merging the above dataframes.
This is NOT the same number as the number of hospitals having real HAC scores.
Instead, it is the same as the number of hospitals with legit scores for PSI-90 or one or more HAIs.


## Generate SIS-based scores

In [5]:
hais = ['CAUTI', 'CLABSI', 'MRSA', 'CDI']
start_dates = df_yr['Start Date'].unique()

new_df = 0
    
for hai in hais:
    
    tdf2 = df_yr[~df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf2.shape[0] > 0:
            
        derived_sis = tdf2[hai + ' SIS'].tolist()
        reported_scores = tdf2[hai + ' Score'].tolist()
        derived_scores = []

        for ii, sis in enumerate(derived_sis):
            sis = float(sis)
            if np.isnan(sis) == True or np.isnan(reported_scores[ii]) == True:
                derived_scores.append(np.nan)
            elif hai == 'CLABSI':
                score = np.nan
                if sis <= clabsi_sis_deciles[0]:
                    score = 1
                elif sis <= clabsi_sis_deciles[1]:
                    score = 2
                elif sis <= clabsi_sis_deciles[2]:
                    score = 3
                elif sis <= clabsi_sis_deciles[3]:
                    score = 4
                elif sis <= clabsi_sis_deciles[4]:
                    score = 5
                elif sis <= clabsi_sis_deciles[5]:
                    score = 6
                elif sis <= clabsi_sis_deciles[6]:
                    score = 7
                elif sis <= clabsi_sis_deciles[7]:
                    score = 8
                elif sis <= clabsi_sis_deciles[8]:
                    score = 9
                elif sis > clabsi_sis_deciles[8]:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
            elif hai == 'CAUTI':
                score = np.nan
                if sis <= cauti_sis_deciles[0]:
                    score = 1
                elif sis <= cauti_sis_deciles[1]:
                    score = 2
                elif sis <= cauti_sis_deciles[2]:
                    score = 3
                elif sis <= cauti_sis_deciles[3]:
                    score = 4
                elif sis <= cauti_sis_deciles[4]:
                    score = 5
                elif sis <= cauti_sis_deciles[5]:
                    score = 6
                elif sis <= cauti_sis_deciles[6]:
                    score = 7
                elif sis <= cauti_sis_deciles[7]:
                    score = 8
                elif sis <= cauti_sis_deciles[8]:
                    score = 9
                elif sis > cauti_sis_deciles[8]:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
            
            elif hai == 'MRSA':
                score = np.nan
                if sis <= mrsa_sis_deciles[0]:
                    score = 1
                elif sis <= mrsa_sis_deciles[1]:
                    score = 2
                elif sis <= mrsa_sis_deciles[2]:
                    score = 3
                elif sis <= mrsa_sis_deciles[3]:
                    score = 4
                elif sis <= mrsa_sis_deciles[4]:
                    score = 5
                elif sis <= mrsa_sis_deciles[5]:
                    score = 6
                elif sis <= mrsa_sis_deciles[6]:
                    score = 7
                elif sis <= mrsa_sis_deciles[7]:
                    score = 8
                elif sis <= mrsa_sis_deciles[8]:
                    score = 9
                elif sis > mrsa_sis_deciles[8]:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
                
            elif hai == 'CDI':
                score = np.nan
                if sis <= cdi_sis_deciles[0]:
                    score = 1
                elif sis <= cdi_sis_deciles[1]:
                    score = 2
                elif sis <= cdi_sis_deciles[2]:
                    score = 3
                elif sis <= cdi_sis_deciles[3]:
                    score = 4
                elif sis <= cdi_sis_deciles[4]:
                    score = 5
                elif sis <= cdi_sis_deciles[5]:
                    score = 6
                elif sis <= cdi_sis_deciles[6]:
                    score = 7
                elif sis <= cdi_sis_deciles[7]:
                    score = 8
                elif sis <= cdi_sis_deciles[8]:
                    score = 9
                elif sis > cdi_sis_deciles[8]:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)


        tdf2[hai + ' SIS-based score'] = derived_scores
        print('len:', len(derived_scores), len(reported_scores))

    # Assign maximum scores to hospitals with HAI footnote 18 
    tdf3 = df_yr[df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf3.shape[0] > 0:
        tdf3[hai + ' SIS-based score'] = 10
        df_yr = pd.concat([tdf2, tdf3], axis=0)
            
    else:
        df_yr = tdf2.copy(deep=True)


len: 3227 3227
len: 3227 3227
len: 3225 3225
len: 3225 3225


## Generate scores based on random expectations

In [6]:
hais = ['CAUTI', 'CLABSI', 'MRSA', 'CDI']
start_dates = df_yr['Start Date'].unique()

new_df = 0
    
for hai in hais:
    
    tdf2 = df_yr[~df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf2.shape[0] > 0:
            
        derived = tdf2[hai + ' expected O/P'].tolist()
        reported_scores = tdf2[hai + ' Score'].tolist()
        derived_scores = []

        for ii, s in enumerate(derived):
            s = float(s)
            if np.isnan(s) == True or np.isnan(reported_scores[ii]) == True:
                derived_scores.append(np.nan)
            elif hai == 'CLABSI':
                score = np.nan
                if s <= clabsi_rand_sir_deciles[0]:
                    score = 1
                elif s <= clabsi_rand_sir_deciles[1]:
                    score = 2
                elif s <= clabsi_rand_sir_deciles[2]:
                    score = 3
                elif s <= clabsi_rand_sir_deciles[3]:
                    score = 4
                elif s <= clabsi_rand_sir_deciles[4]:
                    score = 5
                elif s <= clabsi_rand_sir_deciles[5]:
                    score = 6
                elif s <= clabsi_rand_sir_deciles[6]:
                    score = 7
                elif s <= clabsi_rand_sir_deciles[7]:
                    score = 8
                elif s <= clabsi_rand_sir_deciles[8]:
                    score = 9
                elif s > clabsi_rand_sir_deciles[8]:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
            elif hai == 'CAUTI':
                score = np.nan
                if s <= cauti_rand_sir_deciles[0]:
                    score = 1
                elif s <= cauti_rand_sir_deciles[1]:
                    score = 2
                elif s <= cauti_rand_sir_deciles[2]:
                    score = 3
                elif s <= cauti_rand_sir_deciles[3]:
                    score = 4
                elif s <= cauti_rand_sir_deciles[4]:
                    score = 5
                elif s <= cauti_rand_sir_deciles[5]:
                    score = 6
                elif s <= cauti_rand_sir_deciles[6]:
                    score = 7
                elif s <= cauti_rand_sir_deciles[7]:
                    score = 8
                elif s <= cauti_rand_sir_deciles[8]:
                    score = 9
                elif s > cauti_rand_sir_deciles[8]:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
            
            elif hai == 'MRSA':
                score = np.nan
                if sis <= mrsa_rand_sir_deciles[0]:
                    score = 1
                elif s <= mrsa_rand_sir_deciles[1]:
                    score = 2
                elif s <= mrsa_rand_sir_deciles[2]:
                    score = 3
                elif s <= mrsa_rand_sir_deciles[3]:
                    score = 4
                elif s <= mrsa_rand_sir_deciles[4]:
                    score = 5
                elif s <= mrsa_rand_sir_deciles[5]:
                    score = 6
                elif s <= mrsa_rand_sir_deciles[6]:
                    score = 7
                elif s <= mrsa_rand_sir_deciles[7]:
                    score = 8
                elif s <= mrsa_rand_sir_deciles[8]:
                    score = 9
                elif s > mrsa_rand_sir_deciles[8]:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
                
            elif hai == 'CDI':
                score = np.nan
                if s <= cdi_rand_sir_deciles[0]:
                    score = 1
                elif s <= cdi_rand_sir_deciles[1]:
                    score = 2
                elif s <= cdi_rand_sir_deciles[2]:
                    score = 3
                elif s <= cdi_rand_sir_deciles[3]:
                    score = 4
                elif s <= cdi_rand_sir_deciles[4]:
                    score = 5
                elif s <= cdi_rand_sir_deciles[5]:
                    score = 6
                elif s <= cdi_rand_sir_deciles[6]:
                    score = 7
                elif s <= cdi_rand_sir_deciles[7]:
                    score = 8
                elif s <= cdi_rand_sir_deciles[8]:
                    score = 9
                elif s > cdi_rand_sir_deciles[8]:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)


        tdf2[hai + ' random-based score'] = derived_scores
        print('len:', len(derived_scores), len(reported_scores))

    # Assign maximum scores to hospitals with HAI footnote 18 
    tdf3 = df_yr[df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf3.shape[0] > 0:
        tdf3[hai + ' random-based score'] = 10
        df_yr = pd.concat([tdf2, tdf3], axis=0)
            
    else:
        df_yr = tdf2.copy(deep=True)


len: 3227 3227
len: 3227 3227
len: 3225 3225
len: 3225 3225


In [7]:
holdout_df = df_yr[(df_yr['State'] == 'MD') | (~df_yr['Payment Reduction'].isin(['Yes', 'No'])) | (df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]
df_yr = df_yr[(df_yr['State'] != 'MD') & (df_yr['Payment Reduction'].isin(['Yes', 'No'])) & (~df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]

print(holdout_df.shape[0])
print(df_yr.shape[0])
print(holdout_df.shape[0] + df_yr.shape[0])


46
3187
3233


## Generate SIS-based HAC scores and penalty assignments

In [8]:
hac_scores = []
ct1 = 0
ct2 = 0

for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    d1 = 0
    d2 = 0
        
    w_ls = []
    sum_ls = []

    # Use original scores for all HAIs for a specific type of testing, i.e., can penalties be reproduced when using data directly from the HAC files
    m_ls = ['CAUTI SIS-based score', 'CLABSI SIS-based score', 'MRSA SIS-based score', 'CDI SIS-based score', 'SSI Score']
    
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d2 = np.nan
    else:
        d2 = s/w
        
    d1 = float(tdf['AHRQ PSI-90 Score'].iloc[0])
        
    d2_o = float(tdf['Domain 2 Score'].iloc[0])
    state = tdf['State'].iloc[0]
    
    if np.isnan(d1) == True: 
        # if no score for Domain 1, then total HAC score will be based entirely on Domain 2
        if np.isnan(d2) == True:
            hac_scores.append(np.nan)
        
        elif np.isnan(d2) == False:
            hac_scores.append(d2)
        
    elif np.isnan(d1) == False:
        # if there is a score for Domain 1 ...
        
        # If the derived score is NaN
        if np.isnan(d2) == True:
            hac_scores.append(d1)
        
        # If the derived score is a float
        elif np.isnan(d2) == False:
            hac_scores.append(0.15*d1 + 0.85*d2)
        
    else:
        print(d1, ',', d2, ',', d2_o)
        break

print('ct1:', ct1)
print('ct2:', ct2)




df_yr['Total HAC Score (SIS-based)'] = hac_scores
print(df_yr.shape[0], 'hospitals in hac_df')

tdf = df_yr[~df_yr['Total HAC Score (SIS-based)'].isin([np.nan, float('NaN')])]
p75 = np.percentile(tdf['Total HAC Score (SIS-based)'], 75, method='linear')

pr = []
for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    p = tdf['Payment Reduction'].iloc[0]
    if p != 'Yes' and p != 'No' and np.isnan(p) == True:
        pr.append(np.nan)

    else:
        score = tdf['Total HAC Score (SIS-based)'].iloc[0]

        if np.isnan(score) == True:
            pr.append('No')
        elif score <= p75:
            pr.append('No')
        elif score > p75:
            pr.append('Yes')
        else:
            print('This score is an error:', score)
            sys.exit()
            

df_yr['Payment Reduction (SIS-based)'] = pr
            
o_list = df_yr['Payment Reduction'].tolist()
d_list = df_yr['Payment Reduction (SIS-based)'].tolist()

same = 0
diff = 0
p_to_np = 0
np_to_p = 0

res_ls = []
for i, o in enumerate(o_list):
    if o == d_list[i]:
        same += 1
        res_ls.append(1)
    else:
        diff += 1
        if o == 'Yes' and d_list[i] == 'No':
            p_to_np += 1
        elif o == 'No' and d_list[i] == 'Yes':
            np_to_p += 1
        else:
            print('Error')
            break
        res_ls.append(0)
               
print(same, "SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments.")
print(diff, "SIR-based penalty assignments were NOT the same as their corresponding SIS-based penalty assignments.")
print(str(np.round(100 * same/(same+diff),2)) + '% of SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments."\n')

print(p_to_np, 'hospitals were penalized but should not have been.')
print(np_to_p, 'hospitals were NOT penalized but should have been.')


ct1: 0
ct2: 0
3187 hospitals in hac_df
2746 SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments.
441 SIR-based penalty assignments were NOT the same as their corresponding SIS-based penalty assignments.
86.16% of SIR-based penalty assignments were the same as their corresponding SIS-based penalty assignments."

202 hospitals were penalized but should not have been.
239 hospitals were NOT penalized but should have been.


In [9]:
hac_scores = []
ct1 = 0
ct2 = 0

for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    d1 = 0
    d2 = 0
        
    w_ls = []
    sum_ls = []

    # Use original scores for all HAIs for a specific type of testing, i.e., can penalties be reproduced when using data directly from the HAC files
    m_ls = ['CAUTI random-based score', 'CLABSI random-based score', 'MRSA random-based score', 'CDI random-based score', 'SSI Score']
    
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d2 = np.nan
    else:
        d2 = s/w
        
    d1 = float(tdf['AHRQ PSI-90 Score'].iloc[0])
        
    d2_o = float(tdf['Domain 2 Score'].iloc[0])
    state = tdf['State'].iloc[0]
    
    if np.isnan(d1) == True: 
        # if no score for Domain 1, then total HAC score will be based entirely on Domain 2
        if np.isnan(d2) == True:
            hac_scores.append(np.nan)
        
        elif np.isnan(d2) == False:
            hac_scores.append(d2)
        
    elif np.isnan(d1) == False:
        # if there is a score for Domain 1 ...
        
        # If the derived score is NaN
        if np.isnan(d2) == True:
            hac_scores.append(d1)
        
        # If the derived score is a float
        elif np.isnan(d2) == False:
            hac_scores.append(0.15*d1 + 0.85*d2)
        
    else:
        print(d1, ',', d2, ',', d2_o)
        break

print('ct1:', ct1)
print('ct2:', ct2)




df_yr['Total HAC Score (random-based)'] = hac_scores
print(df_yr.shape[0], 'hospitals in hac_df')

tdf = df_yr[~df_yr['Total HAC Score (random-based)'].isin([np.nan, float('NaN')])]
p75 = np.percentile(tdf['Total HAC Score (random-based)'], 75, method='linear')

pr = []
for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    p = tdf['Payment Reduction'].iloc[0]
    if p != 'Yes' and p != 'No' and np.isnan(p) == True:
        pr.append(np.nan)

    else:
        score = tdf['Total HAC Score (random-based)'].iloc[0]

        if np.isnan(score) == True:
            pr.append('No')
        elif score <= p75:
            pr.append('No')
        elif score > p75:
            pr.append('Yes')
        else:
            print('This score is an error:', score)
            sys.exit()
            

df_yr['Payment Reduction (random-based)'] = pr
            

ct1: 0
ct2: 0
3187 hospitals in hac_df


In [10]:
## SIS-based

tdf1 = df_yr[df_yr['Payment Reduction (SIS-based)'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction (SIS-based)'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction (SIS-based)'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])
print(tdf2.shape[0], len(tdf2['Facility ID'].unique()))
print('\n')

## Random-based 
tdf1 = df_yr[df_yr['Payment Reduction (random-based)'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction (random-based)'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction (random-based)'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])
print(tdf2.shape[0], len(tdf2['Facility ID'].unique()))
print('\n')

## Original payment reduction
tdf1 = df_yr[df_yr['Payment Reduction'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])
print(df_yr.shape[0], len(df_yr['Facility ID'].unique()))

0.24788202070913085
0.7521179792908692
3187 3187


0.24913711954816442
0.7508628804518356
3187 3187


0.2362723564480703
0.7637276435519297
3187 3187


In [11]:
ls1 = list(df_yr)
ls2 = list(holdout_df)
ls = list(filter(lambda x:x in ls1, ls2))
print(df_yr.shape)
print(holdout_df.shape)
df_yr = df_yr.merge(holdout_df, how='outer', on=ls)
print(df_yr.shape)

(3187, 144)
(46, 140)
(3233, 144)


In [12]:
p_np_df = df_yr[(df_yr['Payment Reduction'] == 'Yes') & (df_yr['Payment Reduction (SIS-based)'] == 'No')]
ip_ = np.round(np.nansum(p_np_df['HAC penalty, final']))
print(p_np_df.shape[0])

np_p_df = df_yr[(df_yr['Payment Reduction'] == 'No') & (df_yr['Payment Reduction (SIS-based)'] == 'Yes')]
is_ = np.round(np.nansum(np_p_df['HAC penalty, final']))
print(np_p_df.shape[0])

print(ip_, 'dollars of inappropriate penalties')
print(is_, 'dollars of inappropriate hospital savings')
print('', ip_ - is_)


202
239
158967516.0 dollars of inappropriate penalties
29909427.0 dollars of inappropriate hospital savings
 129058089.0


In [13]:
df_yr.to_pickle('~/GitHub/HACRP-HAIs/data/finalized/final_2017.pkl', protocol=5)